# 朴素贝叶斯 Naive Bayes
优点：在数据较少的情况下仍然有效，可以处理多类别问题。

缺点：对于输入数据的准备方式较为敏感。

适用数据类型：标称型数据。

我们现在用p1(x,y)表示数据点(x,y)属于类别1（图中用圆点表 示的类别）的概率，用p2(x,y)表示数据点(x,y)属于类别2（图中用三角形表示的类别）的概率， 那么对于一个新数据点(x,y)，可以用下面的规则来判断它的类别：

如果 p1(x,y) > p2(x,y)，那么类别为1。如果 p2(x,y) > p1(x,y)，那么类别为2。我们会选择高概率对应的类别。这就是贝叶斯决策理论的核心思想，即选择具有 最高概率的决策。接下来，我们必须要详述p1及p1概率计算方法。为了能够计算p1与p2，有必要讨论一下条件概率

$$\text { P (A | B) }=\mathrm{P}(\text { A and B) } / \mathrm{P}(\text {  } \mathrm{B})$$

贝叶斯准则, 如果已知P(x|c)，要求P(c|x)

$$p(c | x)=\frac{p(x | c) p(c)}{p(x)}$$

前面的两个准则并不是贝叶斯决策理论的所有内容。使用p1( )和p2( )只是为了尽可能简化 描述，而真正需要计算和比较的是p(c 1 |x, y)和p(c 2 |x, y)。这些符号所代表的具体意义是： 给定某个由x、y表示的数据点，那么该数据点来自类别c 1 的概率是多少？数据点来自类别c 2 的概 率又是多少？注意这些概率与刚才给出的概率p(x, y|c 1 )并不一样，不过可以使用贝叶斯准则 来交换概率中条件与结果。具体地，应用贝叶斯准则得到：
$$p\left(c_{i} | x, y\right)=\frac{p\left(x, y | c_{i}\right) p\left(c_{i}\right)}{p(x, y)}$$ 这里将前面的x变成了x，y

使用这些定义，可以定义贝叶斯分类准则为：如果$P(c_1 |x, y) > P(c_2 |x, y)$，那么属于类别$c_1$ ；如果$P(c_1 |x, y) < P(c_2 |x, y)$，那么属于类别$c_2$ ；

由统计学知，如果每个特征需要N个样本，那么对于10个特征将需要 $N^{10}$ 个样本，对于包含1000个特征的词汇表将需要$N^{1000}$ 个样本。可以看到，所需要的样本数会随 着特征数目增大而迅速增长。**如果特征之间相互独立，那么样本数就可以从$N^{1000}$ 减少到1000×N。所谓独立（independence） 指的是统计意义上的独立，即一个特征或者单词出现的可能性与它和其他单词相邻没有关系**.这个假设正 是朴素贝叶斯分类器中朴素（naive）一词的含义。朴素贝叶斯分类器中的另一个假设是，**每个特征同等重要。其实这个假设也有问题**。尽管上述假设存在一些小的 瑕疵，但朴素贝叶斯的实际效果却很好。

#### 朴素贝叶斯的一般过程
(1) 收集数据：可以使用任何方法。本章使用RSS源。

(2) 准备数据：需要数值型或者布尔型数据。

(3) 分析数据：有大量特征时，绘制特征作用不大，此时使用直方图效果更好。

(4) 训练算法：计算不同的独立特征的条件概率。

(5) 测试算法：计算错误率。

(6) 使用算法：一个常见的朴素贝叶斯应用是文档分类。可以在任意的分类场景中使用朴素贝叶斯分类器，不一定非要是文本。

### 使用朴素贝叶斯进行文档分类

朴素贝叶斯分类器通常有两种实现方式：一种基于贝努利模型实现，一种基于多项式模型实现。这里采用前一种 实现方式。该实现方式中并不考虑词在文档中出现的次数，只考虑出不出现，因此在这个意义上相当于假设词是 等权重的。4.5.4节给出的实际上是多项式模型，它考虑词在文档中的出现次数

In [1]:
import bayes
listOfPost, listClass = bayes.loadDataSet()
myVocList = bayes.createVocabList(listOfPost)
print(myVocList)
print(bayes.setOfWords2Vec(myVocList, listOfPost[0]))

['maybe', 'worthless', 'has', 'him', 'I', 'is', 'how', 'garbage', 'steak', 'quit', 'mr', 'dog', 'to', 'dalmation', 'posting', 'help', 'stop', 'take', 'flea', 'park', 'not', 'cute', 'stupid', 'ate', 'love', 'food', 'please', 'licks', 'my', 'problems', 'so', 'buying']
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]


### 训练算法：从词向量计算概率

我们重写贝叶斯准则，将之前的x、y 替换为w。粗体w表示这是一个向量，即它由多个数 值组成。在这个例子中，数值个数与词汇表中的词个数相同。
$$p\left(c_{i} | \boldsymbol{w}\right)=\frac{p\left(\boldsymbol{w} | c_{i}\right) p\left(c_{i}\right)}{p(\boldsymbol{w})}$$

我们将使用上述公式，对每个类计算该值，然后比较这两个概率值的大小。如何计算呢？

1. 首先可以通过类别$i$（侮辱性留言或非侮辱性留言）中文档数除以总的文档数来计算概率$p(c_i)$。

2. 接下来计算$p(w|c_i)$，这里就要用到朴素贝叶斯假设。如果将w展开为一个个独立特征，那么就可以将上述概率写作$p(w_0 , w_1 , w_2 ..w_N | c_i)$。这里假设所有词都互相独立，该假设也称作条件独立性假设，它意味着可以使用$ p(w_0 |c_i) p(w_1 |c_i ) p(w_2 |c_i )...p(w_N |c_i )$来计算上述概率，这就极大地 简化了计算的过程。

3. 因为分母$p_w$ 是一样的，可以不用计算出来

该函数的伪代码如下：

        计算每个类别中的文档数目 
        对每篇训练文档： 
            对每个类别： 
                如果词条出现在文档中→ 增加该词条的计数值 
                增加所有词条的计数值 
        对每个类别： 
            对每个词条： 
                将该词条的数目除以总词条数目得到条件概率 
        返回每个类别的条件概率
        
 利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概 率，即计算$ p(w_0 |c_i) p(w_1 |c_i ) p(w_2 |c_i )...p(w_N |c_i )$, 如果其中一个概率值为0，那么最后的乘积也为0。为降低这种影响，可以将所有词的出现数初始化为1，并将分母初始化为2。
 
 另一个遇到的问题是下溢出，这是由于太多很小的数相乘造成的。当计算乘积 $ p(w_0 |c_i) p(w_1 |c_i ) p(w_2 |c_i )...p(w_N |c_i )$时，由于大部分因子都非常小，所以程序会下溢出或者得到不正确的答案。（读者可以用Python尝试相乘许多很小的数，最后四舍五入后会得到0。）一种解决办法是对乘积取自然对数。在代数中有$ln(a*b) = ln(a)+ln(b)$，于是通过求对数可以 避免下溢出或者浮点数舍入导致的错误。同时，采用自然对数进行处理不会有任何损失。图4-4 给出函数f(x)与ln(f(x))的曲线。检查这两条曲线，就会发现它们在相同区域内同时增加或者减少，并且在相同点上取到极值。它们的取值虽然不同，但不影响最终结果。

In [9]:
trainMat = []
for postinDoc in listOfPost:
  trainMat.append(bayes.setOfWords2Vec(myVocList, postinDoc))
p0V, p1V, pAb = bayes.trainNB0(trainMat,  listClass)
print(pAb) #任意文档属于侮辱性文档的概率
print(p0V)

0.5
[-2.15948425 -2.56494936 -2.56494936 -3.25809654 -2.56494936 -2.56494936
 -3.25809654 -3.25809654 -3.25809654 -2.56494936 -3.25809654 -2.56494936
 -3.25809654 -3.25809654 -2.56494936 -3.25809654 -2.56494936 -2.56494936
 -1.87180218 -3.25809654 -2.56494936 -3.25809654 -2.56494936 -3.25809654
 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936
 -2.56494936 -2.56494936]


### 预测 
前面计算出来了所有的 $ p(w_0 |c_i) p(w_1 |c_i ) p(w_2 |c_i )...p(w_N |c_i )$ 以及 p(c_i)

$$p\left(c_{i} |w\right)=\frac{p\left(w | c_{i}\right) p\left(c_{i}\right)}{p(w)}$$ 

这里忽略了分子$$p\left(w | c_{i}\right) p\left(c_{i}\right) = p(w_0 |c_i) p(w_1 |c_i ) p(w_2 |c_i )...p(w_N |c_i )$$

这里的$p\left(w_i | c_{i}\right)$ 是类别是 $c_{i}$且  $w_i$ 出现的概率，所及预测的时候，只考虑wi 出现的$p\left(w_i | c_{i}\right)$， 同时对两边用log函数，所以代码中是 
p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)

**个人感觉这样只即计算了 某个类别下，wi=1 对应的 $p(w_i |c_i)$相乘， 还应该计算wi=0 对应的 $p(w_i |c_i)$  代码只计算了 $p(w_i = 1 |c_i = 1)$  和 $p(w_i = 1 |c_i = 0)$ 没有计算 $p(w_i = 0 |c_i = 1)$  和 $p(w_i = 0 |c_i = 0)$**

In [10]:
bayes.testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


### 准备数据：文档词袋模型

目前为止，我们将每个词的出现与否作为一个特征，这可以被描述为**词集模型（set-of-words model）**。如果一个词在文档中出现不止一次，这可能意味着包含该词是否出现在文档中所不能表达的某种信息，这种方法被称为词袋模型（bag-of-words model）。在词袋中，每个单词可以出现多次，而在词集中，每个词只能出现一次。为适应词袋模型，需要对函数setOfWords2Vec() 稍加修改，修改后的函数称为bagOfWords2Vec()。

#### 准备数据：切分文本
对于一个文本字符串，可以使用Python的string.split()方法将其切分。下面看看实际的 运行效果。在Python提示符下输入：

In [15]:
mySent = 'This book is the sdis , sdis, d ds'
mySent.split()

['This', 'book', 'is', 'the', 'sdis', ',', 'sdis,', 'd', 'ds']

可以看到，切分的结果不错，但是标点符号也被当成了词的一部分。可以使用正则表示式来切分 句子，其中分隔符是除单词、数字外的任意字符串。

In [16]:
import re

regEx = re.compile(r'\W+')
lst = regEx.split(mySent)
print(lst)

['This', 'book', 'is', 'the', 'sdis', 'sdis', 'd', 'ds']


现在得到了一系列词组成的词表，但是里面的空字符串需要去掉。可以计算每个字符串的长度， 只返回长度大于0的字符串。

In [17]:
[tok for tok in lst if len(tok)>0]

['This', 'book', 'is', 'the', 'sdis', 'sdis', 'd', 'ds']

最后，我们发现句子中的第一个单词是大写的。如果目的是句子查找，那么这个特点会很有用。 但这里的文本只看成词袋，所以我们希望所有词的形式都是统一的，不论它们出现在句子中间、 结尾还是开头.Python中有一些内嵌的方法，可以将字符串全部转换成小写（.lower()）或者大写（.upper()）， 借助这些方法可以达到目的。于是，可以进行如下处理：

In [21]:
[tok.lower() for tok in lst if len(tok)>0]

['this', 'book', 'is', 'the', 'sdis', 'sdis', 'd', 'ds']

现在来看数据集中一封完整的电子邮件的实际处理结果。该数据集放在email文件夹中，该 文件夹又包含两个子文件夹，分别是spam与ham。

In [7]:
bayes.spamTest()

the error rate is:  0.0


In [3]:
bayes.spamTest()

the error rate is:  0.0


### 小结
对于分类而言，使用概率有时要比使用硬规则更为有效。贝叶斯概率及贝叶斯准则提供了一 种利用已知值来估计未知概率的有效方法。

可以通过特征之间的条件独立性假设，降低对数据量的需求。独立性假设是指一个词的出现 概率并不依赖于文档中的其他词。当然我们也知道这个假设过于简单。这就是之所以称为朴素贝 叶斯的原因。尽管条件独立性假设并不正确，但是朴素贝叶斯仍然是一种有效的分类器。

利用现代编程语言来实现朴素贝叶斯时需要考虑很多实际因素。下溢出就是其中一个问题， 它可以通过对概率取对数来解决。词袋模型在解决文档分类问题上比词集模型有所提高。还有其 他一些方面的改进，比如说移除停用词，当然也可以花大量时间对切分器进行优化。

本章学习到的概率理论将在后续章节中用到，另外本章也给出了有关贝叶斯概率理论全面具 体的介绍。接下来的一章将暂时不再讨论概率理论这一话题，介绍另一种称作Logistic回归的分 类方法及一些优化算法。